# Lab Exercise 4: Nyquist signaling and L-ASK
After studying Chapter 4 of the notes and, in particular, Example 4.2, write and run
code for the following:

## Setup

```{admonition} Live Code
Press the following button to make python code interactive. It will connect you to a kernel once it says "ready" (might take a bit, especially the first time it runs).
```

<div style="text-align: center;">
  <button title="Launch thebe" class="thebelab-button thebe-launch-button" onclick="initThebe()">Python Interactive Code</button>
</div>

#### Importing packages we will need later in Python

In [92]:
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.signal
import scipy.special
from scipy.special import erfc
from math import log, log2, sqrt
from ipywidgets import IntSlider, FloatSlider, Button, Layout, interactive

## Part 1: Useful Code

`````` {dropdown} Useful functions
````` {tab} Python
````python

# The function grayCode accepts a number n and returns an array g containing
# the Gray coding for numbers with n bits.
# This is done by recursively calling the function gray_code_recurse
def grayCode(n):
    def gray_code_recurse (g,n):
        k=len(g)
        if n<=0:
            return
        else:
            for i in range (k-1,-1,-1):
                char='1'+g[i]
                g.append(char)
            for i in range (k-1,-1,-1):
                g[i]='0'+g[i]

            gray_code_recurse (g,n-1)

    g=['0','1']
    gray_code_recurse(g,n-1)
    return g


# The function naturalBinaryCoding accepts a number n and returns an array
# with binary numbers with n bits (or equivalently up to the number 2**n, which results
# by left shifting 1 by n bits)
def naturalBinaryCoding(n):
    binary_levels = []
    for i in range(1 << n):
        binary_levels.append('{:0{}b}'.format(i, n))
    return binary_levels


# The function generateRandomBits generates n_bits random binary digits
def generateRandomBits(n_bits):
    bitstream = []
    for i in range(n_bits):
        random_bit = random.randint(0, 1)
        bitstream.append(random_bit)
    return bitstream


# The function createLevels divides the range [-A, A] into L-1 equal intervals,
# so that it always contains the endpoints of the range
def createLevels(A, L):
    y = []
    step = (A - (-A)) / (L - 1)
    for i in range(L):
        y.append(-A + i * step)
    return y


# The function createSymbols takes an array with bits as an argument and
# groups neighboring digits into symbols with length k
def createSymbols(k, bitstream):
    n_bits = len(bitstream)
    symbols = []
    for i in range(0, n_bits - k + 1, k):
        symbol = ""
        for j in range(k):
            symbol += str(bitstream[i+j])
        symbols.append(symbol)
    return symbols


# The function rootRaisedCosine creates a root raised cosine pulse
# with a roll-off coefficient and order determined by the sampling rate (nsample)
# and the delay it will introduce (delay)
def rootRaisedCosine(nsamp, roll_off, delay):
    F0 = 0.5 / nsamp
    Fd = 1
    Fs = Fd * nsamp
    Td = 1 / Fd
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off* roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted) 
            h.append(factor1 * (factor2 + factor3))


    plt.plot(t,h)
    plt.grid()
    plt.title('Root raised cosine filter of order %d and roll-off factor %.1f' % (filter_order, roll_off))
    plt.show()
    
    return h


# The function upSample increases the number of samples of a signal signal by adding nsamp-1
# zeros after each sample of the signal
def upSample(signal, nsamp):
    upSampled = []
    for i in range(len(signal) * nsamp):
        if i % nsamp == 0:
            upSampled.append(signal[i // nsamp])
        else:
            upSampled.append(0)
    return upSampled


# The function downSample reduces the sampling frequency of a signal signal by a factor of nsamp
# by keeping only the samples that are multiples of nsamp (0, nsamp, 2*nsamp, ...)
def downSample(signal, nsamp):
    downSampled = []
    for i in range(0, len(signal), nsamp):
        downSampled.append(signal[i])
        
    return downSampled


# Adds white Gaussian noise with mean value μ (mu) and variance σ^2 (sigma)
def generateAWGN(signal, mu, sigma):
    noise = sigma * np.random.randn(len(signal)) + mu
    return noise
    
````
`````
``````

## Part 2: Signal generation with Nyquist filters – Time and frequency plots
Generate a random binary sequence of 10000 bits and then a corresponding 8-ASK
baseband signal, with the following characteristics:

- Gray coding
- Nyquist root raised cosine signaling, with roll-off =0.40
- Over-sampling with nsamp=20 samples per base period T
- Transmitter filter class: 200 (10 periods, group_delay=5T)

``` {hint} 
To Gray encode a binary vector, x, into L-ASK symbols, use the following code segment,
where step is the distance (or increment) between adjacent L-ASK points.
Briefly explain its function and, in particular, the role of the mapping vector (see also Hint in the
next question).
```

```` {tab} Matlab
```matlab

L=…; step=…; % ο αριθμός των πλατών και το βήμα μεταξύ τους
k=log(L);
mapping=[step/2; -step/2];
if(k>1)
	for j=2:k
	mapping=[mapping+2^(j-1)*step/2; ...
	-mapping-2^(j-1)*step/2];
	end
end;
xsym=bi2de(reshape(x,k,length(x)/k).','left-msb');
y=[];
for i=1:length(xsym)
	y=[y mapping(xsym(i)+1)];
end

```
````

1. Calculate the signal at the output of the matched filter at the receiver.
Show part of this signal (with the plot command) of 10T duration.
2. Superimpose (with the stem command) on this part the corresponding
samples of the input signal in the basic period grid T (before over-sampling)
3. Plot (using the pwelch command) the spectrum of the signal at the
receiver and explain its shape and amplitude.

In [95]:
# The function grayCode accepts a number n and returns an array g containing
# the Gray coding for numbers with n bits.
# This is done by recursively calling the function gray_code_recurse
def grayCode(n):
    def gray_code_recurse (g,n):
        k=len(g)
        if n<=0:
            return
        else:
            for i in range (k-1,-1,-1):
                char='1'+g[i]
                g.append(char)
            for i in range (k-1,-1,-1):
                g[i]='0'+g[i]

            gray_code_recurse (g,n-1)

    g=['0','1']
    gray_code_recurse(g,n-1)
    return g


# The function naturalBinaryCoding accepts a number n and returns an array
# with binary numbers with n bits (or equivalently up to the number 2**n, which results
# by left shifting 1 by n bits)
def naturalBinaryCoding(n):
    binary_levels = []
    for i in range(1 << n):
        binary_levels.append('{:0{}b}'.format(i, n))
    return binary_levels


# The function generateRandomBits generates n_bits random binary digits
def generateRandomBits(n_bits):
    bitstream = []
    for i in range(n_bits):
        random_bit = random.randint(0, 1)
        bitstream.append(random_bit)
    return bitstream


# The function createLevels divides the range [-A, A] into L-1 equal intervals,
# so that it always contains the endpoints of the range
def createLevels(A, L):
    y = []
    step = (A - (-A)) / (L - 1)
    for i in range(L):
        y.append(-A + i * step)
    return y


# The function createSymbols takes an array with bits as an argument and
# groups neighboring digits into symbols with length k
def createSymbols(k, bitstream):
    n_bits = len(bitstream)
    symbols = []
    for i in range(0, n_bits - k + 1, k):
        symbol = ""
        for j in range(k):
            symbol += str(bitstream[i+j])
        symbols.append(symbol)
    return symbols


# The function rootRaisedCosine creates a root raised cosine pulse
# with a roll-off coefficient and order determined by the sampling rate (nsample)
# and the delay it will introduce (delay)
def rootRaisedCosine(nsamp, roll_off, delay):
    F0 = 0.5 / nsamp
    Fd = 1
    Fs = Fd * nsamp
    Td = 1 / Fd
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off* roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted) 
            h.append(factor1 * (factor2 + factor3))
    
    return t,h


# The function upSample increases the number of samples of a signal signal by adding nsamp-1
# zeros after each sample of the signal
def upSample(signal, nsamp):
    upSampled = []
    for i in range(len(signal) * nsamp):
        if i % nsamp == 0:
            upSampled.append(signal[i // nsamp])
        else:
            upSampled.append(0)
    return upSampled


# The function downSample reduces the sampling frequency of a signal signal by a factor of nsamp
# by keeping only the samples that are multiples of nsamp (0, nsamp, 2*nsamp, ...)
def downSample(signal, nsamp):
    downSampled = []
    for i in range(0, len(signal), nsamp):
        downSampled.append(signal[i])
        
    return downSampled


# Adds white Gaussian noise with mean value μ (mu) and variance σ^2 (sigma)
def generateAWGN(signal, mu, sigma):
    noise = sigma * np.random.randn(len(signal)) + mu
    return noise


global filt, g_delay, g_nsamp
def interactive_signal_processing(n_bits, L, roll_off, nsamp, delay):
    global filt, g_delay, g_nsamp
    g_delay = delay
    g_nsamp = nsamp
    bitstream = generateRandomBits(n_bits)
    A = L - 1
    k = int(log2(L))
    y_levels = createLevels(A, L)
    symbols = createSymbols(k, bitstream)
    gray_encoding = grayCode(k)

    x_gray = [y_levels[gray_encoding.index(symbol)] for symbol in symbols]
    t_filter, filt = rootRaisedCosine(nsamp, roll_off, delay)
    y = upSample(x_gray, nsamp)
    y_transmitted = scipy.signal.convolve(y, filt)
    y_received = scipy.signal.convolve(y_transmitted, filt)
    y_final = downSample(y_received, nsamp)
    y_final = y_final[2 * delay: len(y_final) - 2 * delay]

    fig, axs = plt.subplots(1, 3, figsize=(20, 5))
    axs[0].plot(t_filter, filt)
    axs[0].set_title('Root Raised Cosine Filter')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('Amplitude')
    axs[0].grid(True)
    
    t = np.arange(0, len(y[:10]))
    axs[1].plot(t, y_final[:10])
    axs[1].stem(t, x_gray[:10])
    axs[1].set_title('Signal Visualization')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Amplitude')
    axs[1].grid(True)

    f, Pxx_den = scipy.signal.welch(y_received, window='hamming', nperseg=8192)
    Pxx_den = 10 * np.log10(Pxx_den)
    axs[2].plot(f, Pxx_den)
    axs[2].set_title('Power Spectral Density of the Received Signal')
    axs[2].set_xlabel('Normalized Frequency')
    axs[2].set_ylabel('Power/Frequency [dB]')
    axs[2].grid(True)
    plt.tight_layout()
    plt.show()

# Widget setup for interactive adjustments
w = interactive(interactive_signal_processing,
                n_bits=IntSlider(min=10000, max=100000, step=10000, value=10000, description='Bitstream Length', layout=Layout(width='100%')),
                L=IntSlider(min=2, max=16, step=1, value=8, description='ASK Levels', layout=Layout(width='100%')),
                roll_off=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.4, description='Roll-off Factor', layout=Layout(width='100%')),
                nsamp=IntSlider(min=10, max=40, step=1, value=20, description='nsamp', layout=Layout(width='100%')),
                delay=IntSlider(min=1, max=10, step=1, value=5, description='Group Delay', layout=Layout(width='100%')))
display(w)

interactive(children=(IntSlider(value=10000, description='Bitstream Length', layout=Layout(width='100%'), max=…

## Part 3: Performance calculation: BER vs Eb/No. 
  Study of the effect of parameters: Nyquist filter order and roll-off

For 8-ASK, obtain the BER-Eb/No curve theoretically and by simulation
- with roll-off=0.1 and filter order
  1. 80 (4 periods, group_delay =2T)
  2. 160 (8 periods, group_delay =4T)
  3. 320 (16 periods, group_delay =8T).
- with roll-off=0.2 and filter order as in cases a,b,c above.
- with roll-off=0.4 and filter class as in cases a,b,c above

``` {hint}
For Maximum Likelihood estimation, compare each received symbol (sample at the output
of the matched filter) with the elements of the mapping vector. The position of the closest
element in the vector will also give the corresponding codeword. The following code snippet is an
implementation of this forator.
```

```` {tab} Matlab
```matlab

% yr: the received symbol, at the matched filter output
% xr: the encoded bit word, k bits
[m,j]=min(abs(mapping-yr));
xr=de2bi(j-1,k,'left-msb');

```
````

In [94]:
# This function wraps the signal processing and BER calculation, making parameters adjustable via sliders
def interactive_signal_processing(n_bits=80000, L=16, roll_off=0.7, nsamp=16, delay=4):
    bitstream = generateRandomBits(n_bits)
    A = L - 1
    k = int(log2(L))
    y_levels = createLevels(A, L)
    symbols = createSymbols(k, bitstream)
    gray_encoding = grayCode(k)

    x_gray = [y_levels[gray_encoding.index(symbol)] for symbol in symbols]
    t_filter, filt = rootRaisedCosine(nsamp, roll_off, delay)
    y = upSample(x_gray, nsamp)
    y_transmitted = scipy.signal.convolve(y, filt)

    # Calculating BER theoretically and via simulation
    berTheoretical = []
    berSimulation = []
    EbN0_max = 20

    for EbN0_db in range(1, EbN0_max):
        EbN0 = 10 ** (EbN0_db / 10)
        SNR_db = EbN0_db - 10 * np.log10(nsamp / 2 / k)
        SNR = 10 ** (SNR_db / 10)
        P = sum(y_transmitted * y_transmitted) / len(y_transmitted)
        Pn = P / SNR

        noise = np.random.normal(0, sqrt(Pn), len(y_transmitted))
        y_noisy = y_transmitted + noise

        z_received = scipy.signal.convolve(y_noisy, filt)
        z_final = downSample(z_received, nsamp)
        z_final = z_final[2 * delay : len(z_final) - 2 * delay]

        errors = 0
        for i in range(len(z_final)):
            differences = np.abs(y_levels - z_final[i])
            index = np.argmin(differences)
            if y_levels[index] != x_gray[i]:
                errors += 1
        
        berSimulation.append(errors / n_bits)
        Pe = (L - 1) / L * erfc(sqrt(3 * k / (L**2 - 1) * EbN0))
        berTheoretical.append(Pe / k)
    
    # Plotting the BER
    plt.figure(figsize=(10, 6))
    plt.semilogy( berTheoretical, label='Theoretical')
    plt.semilogy(berSimulation, 'o', label='Simulation')
    plt.xlabel('Eb/N0 (dB)')
    plt.ylabel('Bit Error Rate')
    plt.title('BER Curve for L-ASK')
    plt.legend()
    plt.grid(True)
    plt.show()

# Updated function call with full-width sliders
interact(interactive_signal_processing,
         n_bits=IntSlider(min=10000, max=100000, step=10000, value=80000, description='Binary Sequence Length', layout=Layout(width='100%')),
         L=IntSlider(min=2, max=32, step=1, value=16, description='ASK Levels', layout=Layout(width='100%')),
         roll_off=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.7, description='Roll-off', layout=Layout(width='100%')),
         nsamp=IntSlider(min=8, max=32, step=1, value=16, description='nsamp', layout=Layout(width='100%')),
         delay=IntSlider(min=1, max=8, step=1, value=4, description='Group Delay', layout=Layout(width='100%')));

interactive(children=(IntSlider(value=80000, description='Binary Sequence Length', layout=Layout(width='100%')…

## Part 4: The effect of the coding method: Gray or other
Produce BER-Eb/No curves for 16-ASK and 8-ASK with other coding (not Gray), e.g.
with mapping=-(L-1):step:(L-1), and compare them with the theoretical ones.
What do you observe?

## Part 5: Calculation of system parameters
Adjust the parameters of the 16-ASK transmission system to the following actual data/requirements:
- Width of (baseband) bandwidth $W = 1 \text{ MHz}$
- Noise spectrum density $N_0 = 100 \text{ picowatt/Hz}$ (one-sided)
- $5 \text{ Mbps}$ transmission rate
- Tolerable $BER=2 \text{ Kbps}$

Confirm the specifications in bandwidth and BER.

**Connection with theory**: The required (baseband) bandwidth with Nyquist signaling, is equal to: $$ W = \frac{1}{2T}(1+\alpha) $$ where α the roll-off factor of the Nyquist filter and $ \frac{1}{T} $ the symbol transmission rate (also called Baud Rate). On the other hand, the transmission rate, $R$ (bits/s), is related to $ \frac{1}{T} $ and the size of the sign constellation, L, with the relation: $$ \frac{R}{\log_2 L} = \frac{l}{T} $$

$ \frac{1}{T} $ consists the connecting parameter between $W$ and $R$.